In [2]:
using DataFrames
using Plots
using CSV
using Statistics
using DelimitedFiles

szczepienia = DataFrame(CSV.File("vaccination_data.txt"))

mutable struct Person1
    zyje::Bool
    chetny_do_szczepienia::Bool
    zaszczepiony1::Bool
    zaszczepiony2::Bool
    zasz_data_1::Int16 #data jako numer dnia od poczatku trwania pandemii
    zasz_data_2::Int16
    zarazony::Bool
    data_zarazenia::Int16
end



In [3]:
function zaraza(szczepienia, dl_prognozy::Int, chec_szczepienia::Float64)
    """
    funkcja przyjmuje dopasowany zestaw danych, dlugosc prognozy w dniach i chec spoleczenstwa do szczepien. 
    Zwraca tablice o 4 kolumnach z:
    dzienna liczba zarazen, dziennej liczbie smierci, liczbie zaszczepionych 1 i 2 dawka.
    """
    populacja = Int64(maximum(szczepienia[2:end, 22]))
    Polacy = Vector{Person1}(undef, populacja)
    
    for i in 1:populacja
        Polacy[i] = Person1(false, false, false, false, 0, 0 , false, 0)
    end
    
    # Tworzenie przybliżonej obecnej (2021-06-04) sytuacji:
    
    #chec_szczepienia = 0.57 #obecna sytuacja posrod niezaszczepionych osob
    liczba_przypadkow = Int64(szczepienia[457,2])
    liczba_zgonow = Int64(szczepienia[457,4])
    liczba_zaszczepionych1 = Int64(szczepienia[457,14])
    liczba_zaszczepionych2 = Int64(szczepienia[457,15])
    obecni_zarazeni = Int64(sum(szczepienia[450:457,3])) #zalozenie ze obecni zarazeni choruja i roznosza chorobe przez tydzien
    obecni_zarazeni2 = obecni_zarazeni
    
    for i in 1:populacja
        if rand()<(1 - liczba_zgonow/populacja)
            Polacy[i].zyje = true
            if rand()<liczba_zaszczepionych1/populacja
                Polacy[i].chetny_do_szczepienia = true
                Polacy[i].zaszczepiony1 = true
                if rand()<liczba_zaszczepionych2/liczba_zaszczepionych1
                    Polacy[i].zaszczepiony2 = true
                else
                    Polacy[i].zasz_data_1 = rand(422:457) # przyblizony okres pierwszego szczepienia
                end
            else
                if rand()<chec_szczepienia
                    Polacy[i].chetny_do_szczepienia = true
                end
                if obecni_zarazeni2 > 0
                    Polacy[i].zarazony = true
                    Polacy[i].data_zarazenia = rand(450:457)
                    obecni_zarazeni2 -= 1
                end
            end
        end
    end

    # Tworzenie tablicy z danymi na okreslona dlugosc i symulacja zmianiajacej sie sytuacji:
    
    Tablica = zeros(Int64, dl_prognozy+1, 6)
    obecny_dzien = 458
    ozdrowiency = szczepienia[450,3]
    
    #surowosc_obostrzen = szczepienia[457, 21]
    #reprodukcja_wirusa = szczepienia[457, 6]
    
    
                   #dzienne zarazenia         dzienna smierc             liczba_zaszczepionych1  liczba_zaszczepionych2   obecnie zarazeni
    Tablica[1,:] = [Int64(szczepienia[457,3]), Int64(szczepienia[457,5]), liczba_zaszczepionych1, liczba_zaszczepionych2, obecni_zarazeni, ozdrowiency]
    
    #println(obecni_zarazeni) 
    
    for i in 2:dl_prognozy+1
        
        obecni_zarazeni = Tablica[i-1,5]
        szansa_kontaktu_z_zarazonym = 0.15 * obecni_zarazeni/populacja #zalozenie ze spotykamy 10osob dziennie srednio
        skutecznosc1 = 0.5
        skutecznosc2 = 0.9
        dzisiaj_smierc = 0
        dzisiaj_zarazeni = 0
        dzisiaj_zaszczepieni = 0
        dzisiaj_zaszczepieni2 = 0
        ozdrowiency = 0
        
        
        for i in 1:populacja
            if Polacy[i].zyje
                
                if Polacy[i].zarazony                                        #co z zarazonymi/smierc
                    
                    if Polacy[i].data_zarazenia+7 <= obecny_dzien
                        praw = rand()
                        if praw<0.9 # po 7dniach szansa wyzdrowienia 90% kazdego dnia
                            Polacy[i].zarazony = false
                            Polacy[i].data_zarazenia = 0
                            obecni_zarazeni -= 1
                            ozdrowiency += 1
                        elseif praw<0.927 # zakladamy, ze moga umrzec tylko ci co choruja ponad tydzien
                            """
                            3% umiera, wiec po tym tygodniu zostaje to 30%, zatem szukamy x takiego, ze:
                            0.03 = x * szereg((0.1)^i)
                            czyli wychodzi, ze prawdopodobienstwo smierci po 7dniu choroby to 0.27 kazdego dnia
                            """
                            Polacy[i].zyje = false
                            Polacy[i].data_zarazenia = 0
                            dzisiaj_smierc += 1
                            obecni_zarazeni -= 1
                        end
                    end
                        
                    
                
                else                                                         #szansa zachorowania + zaszczepienia(not yet)
                   
                    
                    prawd = rand() # by przyspieszyc program
                    if Polacy[i].zaszczepiony1
                        if Polacy[i].zaszczepiony2                                       #przy 2dawkach
                            if prawd < szansa_kontaktu_z_zarazonym && rand()>skutecznosc2
                                Polacy[i].zarazony = true
                                Polacy[i].data_zarazenia = obecny_dzien
                                obecni_zarazeni += 1
                                dzisiaj_zarazeni += 1
                            end
                        else                                                             #przy 1dawce
                            if prawd < szansa_kontaktu_z_zarazonym && rand()>skutecznosc1
                                Polacy[i].zarazony = true
                                Polacy[i].data_zarazenia = obecny_dzien
                                obecni_zarazeni += 1
                                dzisiaj_zarazeni += 1
                            elseif Polacy[i].zasz_data_1+35 <= obecny_dzien
                                Polacy[i].zaszczepiony2 = true
                                dzisiaj_zaszczepieni += 1
                                dzisiaj_zaszczepieni2 += 1
                            end
                        end
                    else                                                                 #niezaszczepiony
                        if prawd < szansa_kontaktu_z_zarazonym
                            Polacy[i].zarazony = true
                            Polacy[i].data_zarazenia = obecny_dzien
                            obecni_zarazeni += 1
                            dzisiaj_zarazeni += 1
                        elseif Polacy[i].chetny_do_szczepienia == true && Polacy[i].zasz_data_1 != obecny_dzien
                            Polacy[i].zasz_data_1 = obecny_dzien + rand(15:50)                        
                        elseif Polacy[i].zasz_data_1 == obecny_dzien
                            Polacy[i].zaszczepiony1 = true
                            dzisiaj_zaszczepieni += 1
                        end
                    end
                end
            end
        end
        println("test", i) #dla ogladania szybkosci postepu kodu
        #aktualizacja tablicy:
        Tablica[i,:] = [dzisiaj_zarazeni,dzisiaj_smierc,Tablica[i-1,3]+dzisiaj_zaszczepieni,Tablica[i-1,4]+dzisiaj_zaszczepieni2,obecni_zarazeni, ozdrowiency]
        obecny_dzien += 1 
    end
    
    
    return Tablica
end

zaraza (generic function with 1 method)

Ponizej pierwsza symulacja, w ktorej chec zaszczepienie deklaruje az 90% jeszcze nie zaszczepionych.

In [4]:
x1 = zaraza(szczepienia, 365, 0.90)

test2
test3
test4
test5
test6
test7
test8


LoadError: [91mInterruptException:[39m

In [1]:
y1 = DataFrame(
    dzienne_zarazenia = x1[1:end,1],
    dzienna_smierc = x1[1:end,2],
    lacznie_1_dawka = x1[1:end,3],
    lacznie_2_dawka = x1[1:end,4],
    teraz_zarazeni = x1[1:end,5],
    ozdrowiency = x1[1:end,6]
    )

LoadError: [91mUndefVarError: x not defined[39m

In [21]:
CSV.write("symulacja_90proc.csv", y1)

"symulacja_57proc.csv"

Ponizej druga symulacja, w ktorej chec zaszczepienie deklaruje 10% jeszcze nie zaszczepionych.

In [10]:
x2 = zaraza(szczepienia, 365, 0.10)

LoadError: [91mKeyError: key "y" not found[39m

In [9]:
y2 = DataFrame(
    dzienne_zarazenia = x2[1:end,1],
    dzienna_smierc = x2[1:end,2],
    lacznie_1_dawka = x2[1:end,3],
    lacznie_2_dawka = x2[1:end,4],
    teraz_zarazeni = x2[1:end,5],
    ozdrowiency = x2[1:end,6]
    )

101×6 Array{Int64,2}:
 565  91  14224874   7777048  5642  1202
 592  30  14581755   8133929  4942  1262
 553  26  14760703   8312877  4733   736
 517  23  14938894   8491068  4562   665
 482  24  15117699   8669873  4317   703
 470  27  15295725   8847899  4105   655
 431  23  15474582   9026756  3827   686
 418  21  15653232   9205406  3528   696
 350  20  15832355   9384529  3268   590
 343  17  16010899   9563073  3055   539
 356  13  16189628   9741802  2900   498
 317  16  16368495   9920669  2730   471
 288  16  16547451  10099625  2538   464
   ⋮                                   ⋮
   0   0  20658135  14210309     2     1
   0   0  20658135  14210309     2     0
   0   0  20658135  14210309     1     1
   1   0  20658135  14210309     2     0
   0   0  20658135  14210309     2     0
   0   0  20658135  14210309     1     1
   0   0  20658135  14210309     1     0
   0   0  20658135  14210309     1     0
   0   0  20658135  14210309     1     0
   0   0  20658135  14210309     1 

In [ ]:
CSV.write("symulacja_10proc.csv", y2)

Ponizej trzecia symulacja, w ktorej chec zaszczepienie deklaruje az 40% jeszcze nie zaszczepionych.

In [ ]:
x3 = zaraza(szczepienia, 365, 0.40)

In [ ]:
y3 = DataFrame(
    dzienne_zarazenia = x3[1:end,1],
    dzienna_smierc = x3[1:end,2],
    lacznie_1_dawka = x3[1:end,3],
    lacznie_2_dawka = x3[1:end,4],
    teraz_zarazeni = x3[1:end,5],
    ozdrowiency = x3[1:end,6]
    )

In [ ]:
CSV.write("symulacja_40proc.csv", y3)